In [225]:
import pandas as pd

In [226]:
cols = ['year',
 'abi',
 'ticker',
 'company',
 'address_line_1',
 'city',
 'zipcode',
 'location_employee_size_code',
 'location_sales_volume_code',
 'primary_naics_code',
 'sic_code',
 'sic6_descriptions_sic',
 'business_status_code',
 'office_size_code',
 'company_holding_status',
 'parent_employee_size_code',
 'parent_sales_volume_code',
 'census_tract',
 'cbsa_code',
 'year_established',
 'employee_size_location',
 'sales_volume_location',
 'parent_actual_employee_size',
 'parent_actual_sales_volume',
 'latitude',
 'longitude']

bus = pd.read_csv("../data/chi_bus.csv", sep='\t', names=cols)

In [227]:
bus.shape

(1144876, 26)

In [228]:
# filtering for useful features

bus = bus[['abi','primary_naics_code','company','year','business_status_code','company_holding_status','census_tract','year_established',
    'employee_size_location','sales_volume_location','parent_actual_employee_size','parent_actual_sales_volume',
    'latitude','longitude']]

In [229]:
# understanding business status code

"""
Business Status Code:

1: Headquarter
2: Branch
3: Subsidiary
9: Single Location

"""

bus['business_status_code'].value_counts()

9    1034553
2     101871
3       4241
1       4211
Name: business_status_code, dtype: int64

In [230]:
# under standing parent company data

"""

Info on parent company: Tracts pretty closely with number of companies IDed as branches

"""

bus['parent_actual_employee_size'].isna().value_counts()

True     1043225
False     101651
Name: parent_actual_employee_size, dtype: int64

In [231]:
# understanding business status holding

"""
Business Status Holding:

I'm gonna assume the 712 are publicly traded and the others aren't?

"""

bus['company_holding_status'].isna().value_counts()

True     1144164
False        712
Name: company_holding_status, dtype: int64

In [232]:
# understanding company holding status

bus['company_holding_status'].isna().value_counts()

True     1144164
False        712
Name: company_holding_status, dtype: int64

In [233]:
bus['primary_naics_code'].isna().value_counts()

False    1143647
True        1229
Name: primary_naics_code, dtype: int64

In [234]:
# remove naics code nulls and convert to string

bus = bus[~bus['primary_naics_code'].isna()]
bus['primary_naics_code'] = (bus['primary_naics_code'].astype(int)).astype(str)

In [235]:
# filtering for naics codes

"""

NAICS CODE 445110: Supermarkets and Other Grocery (except Convenience)
NAICS CODE 447110: Gasoline Stations with Convenience Stores
NAICS CODE 445120: Convenience Stores

"""

groc = bus[(bus['primary_naics_code'].str.contains('445110')) | (bus['primary_naics_code'].str.contains('447110')) | (bus['primary_naics_code'].str.contains('445120'))]


In [236]:
groc.shape

(14851, 14)

In [237]:
# understanding year distribution 

groc['year'].value_counts()

2013    1985
2014    1808
2012    1706
2015    1623
2009    1602
2010    1591
2011    1575
2016    1391
2017    1187
2018     383
Name: year, dtype: int64

In [238]:
# filtering for 2016 and 2017

groc_2016 = groc[groc['year']==2016]
groc_2017 = groc[groc['year']==2017]

In [239]:
demand = pd.merge(left=groc_2017, right=groc_2016, left_on='abi', right_on='abi')

In [240]:
demand.shape

(1112, 27)

In [241]:
demand.drop(['primary_naics_code_y','company_y','year_y','business_status_code_y',
            'company_holding_status_y','census_tract_y','year_established_y',
            'company_y','year_y','business_status_code_y',
            'parent_actual_employee_size_y','parent_actual_sales_volume_y',
            'latitude_y','longitude_y'], inplace=True, axis=1)

cols = [demand.columns[i].replace("_x","_2017").replace("_y","_2016") for i in range(len(demand.columns))]
replacing = {i:j for (i,j) in zip(demand.columns,cols)}
demand.rename(columns=replacing, inplace=True)

In [242]:
demand

,abi,primary_naics_code_2017,company_2017,year_2017,business_status_code_2017,company_holding_status_2017,census_tract_2017,year_established_2017,employee_size_location_2017,sales_volume_location_2017,parent_actual_employee_size_2017,parent_actual_sales_volume_2017,latitude_2017,longitude_2017,employee_size_location_2016,sales_volume_location_2016
0,150832897,44511003,BOCKWINKEL'S,2017,9,NaN,320100,NaN,7.0,1789.0,NaN,NaN,41.88488,-87.61501,7.0,1858.0
1,151026986,44511003,Y H PROVISION,2017,9,NaN,833000,NaN,20.0,5111.0,NaN,NaN,41.88456,-87.65530,20.0,5310.0
2,151036118,44511003,GUZMAN GROCERY,2017,9,NaN,841200,NaN,3.0,767.0,NaN,NaN,41.85565,-87.68117,3.0,796.0
3,151053519,44511006,DANIELSON FOOD PRODUCTS,2017,9,NaN,835500,1939.0,20.0,5111.0,NaN,NaN,41.81885,-87.63207,20.0,5310.0
4,151054939,44511003,EL GUERO,2017,9,NaN,841300,1976.0,50.0,12778.0,NaN,NaN,41.85195,-87.67891,50.0,13276.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1107,402005936,44511003,SUN FRESH MARKET & DELI,2017,2,NaN,150600,NaN,10.0,2556.0,1900.0,5100000.0,41.94710,-87.76715,10.0,2655.0
1108,151602547,44512001,STOP-N-GO,2017,2,NaN,750600,NaN,3.0,767.0,25.0,34600.0,41.68977,-87.66263,3.0,796.0
1109,611130287,44512001,TRUENORTH ENERGY,2017,2,NaN,120300,NaN,7.0,1789.0,20.0,992781.0,41.95363,-87.72771,7.0,1858.0
1110,458531563,44512001,SPEEDWAY,2017,2,NaN,841300,NaN,11.0,2812.0,1800.0,63364000.0,41.84973,-87.68519,11.0,2920.0
